In [ ]:
# stdlib
import os
from pathlib import Path

# syft absolute
import syft as sy

In [ ]:
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"
# os.environ["SERVER_PORT"] = "8081"

In [ ]:
environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
server_port = os.environ.get("SERVER_PORT", "auto")
print(environment, server_port)

In [ ]:
server = sy.orchestra.launch(
    name="test_upgradability",
    dev_mode=True,
    reset=False,
    port=server_port,
    migrate=False,
    n_consumers=2,
    create_producer=True,
)

client = server.login(email="info@openmined.org", password="changethis")

In [ ]:
# Check if this server has data on it
assert len(client.users.get_all()) == 2

In [ ]:
migration_data = client.get_migration_data(include_blobs=True)
migration_data

In [ ]:
assert migration_data.includes_blobs
assert migration_data.num_action_objects > 0

In [ ]:
migration_data_dir = Path(os.getenv("MIGRATION_DATA_DIR", "."))
migration_data_dir.mkdir(exist_ok=True)

blob_path = migration_data_dir / "migration.blob"
yaml_path = migration_data_dir / "migration.yaml"

blob_path.unlink(missing_ok=True)
yaml_path.unlink(missing_ok=True)

In [ ]:
migration_data.save(blob_path, yaml_path=yaml_path)

assert blob_path.exists()
assert yaml_path.exists()

print(f"Saved migration data to {str(blob_path.resolve())}")

In [ ]:
if server.server_type.value == "python":
    server.land()